# Test of the new driver for Tektronix 7k AWG

`Found TEKTRONIX AWG7122C Serial: B051135 FW: SCPI:99.0 FW:4.6.0.7 options: ['01', '06', '08', '09']`

## Basic commands and Hardware settings

### Basic commands

#### write()

In [4]:
myawg.write('*RST')

0

In [5]:
myawg.write('OUTPUT1:STATE ON')
myawg.get_errors()

True

In [6]:
myawg.load_waveform({1: '*Ramp10'})
myawg.write('OUTPUT1:STATE ON')

0

In [7]:
myawg.write('OUTPUT1:STATE OFF')

0

#### read()

In [8]:
myawg.write('*IDN?')
myawg.read()

'TEKTRONIX,AWG7122C,B051135,SCPI:99.0 FW:4.6.0.7 '

#### query()

In [10]:
myawg.query('*IDN?')

'TEKTRONIX,AWG7122C,B051135,SCPI:99.0 FW:4.6.0.7'

In [11]:
myawg.write('OUTPUT1:STATE ON')
myawg.query('OUTPUT1:STATE?')

'1'

In [12]:
myawg.write('OUTPUT1:STATE OFF')
myawg.query('OUTPUT1:STATE?')

'0'

#### reset()

In [13]:
myawg.reset()  # works

0

#### get_status()

In [6]:
myawg.pulser_off()
myawg.get_status()

(0, {-1: 'Failed Request or Communication', 0: 'Device has stopped, but can receive commands', 1: 'Device is active and running', 2: 'Device is waiting for trigger.'})

In [5]:
myawg.pulser_on()
myawg.get_status()

(2, {-1: 'Failed Request or Communication', 0: 'Device has stopped, but can receive commands', 1: 'Device is active and running', 2: 'Device is waiting for trigger.'})

#### get_errors()

In [27]:
myawg.get_errors()

False

In [25]:
myawg.get_errors()

True

```
AWG7122C error: -100 "Command error; Command not allowed - *IDN
AWG7122C error: -101 "Invalid character; Unexpected character found - OUTPUT1:STATE ?
```

In [30]:
myawg.set_active_channels({'a_ch2': True})
myawg.get_errors(level='warn')

True

In [32]:
myawg.set_active_channels({'a_ch2': True})
myawg.get_errors()

True

### Hardware settings

#### get_constraints()

In [41]:
constr_obj = myawg.get_constraints()
type(constr_obj)

<class 'interface.pulser_interface.PulserConstraints'>

In [44]:
myawg.set_interleave(False)
constr_obj = myawg.get_constraints()

print('min: {} MHz'.format(constr_obj.sample_rate.min/1e6))
print('max: {} GHz'.format(constr_obj.sample_rate.max/1e9))
print('stp: {} MHz'.format(constr_obj.sample_rate.step/1e6))
print('def: {} GHz'.format(constr_obj.sample_rate.default/1e9))

min: 10.0 MHz
max: 12.0 GHz
stp: 10.0 MHz
def: 12.0 GHz


In [43]:
myawg.set_interleave(True)
constr_obj = myawg.get_constraints()

print('min: {} MHz'.format(constr_obj.sample_rate.min/1e6))
print('max: {} GHz'.format(constr_obj.sample_rate.max/1e9))
print('stp: {} MHz'.format(constr_obj.sample_rate.step/1e6))
print('def: {} GHz'.format(constr_obj.sample_rate.default/1e9))

min: 12000.0 MHz
max: 24.0 GHz
stp: 0.0005 MHz
def: 24.0 GHz


In [45]:
allowed_patterns = constr_obj.activation_config
for pattern_name, pattern_set in allowed_patterns.items():
    print('{}: {}'.format(pattern_name, pattern_set))

: set()
all: {'a_ch1', 'd_ch2', 'd_ch1', 'a_ch2', 'd_ch4', 'd_ch3'}
A1_M1_M2: {'a_ch1', 'd_ch2', 'd_ch1'}
A2_M3_M4: {'d_ch4', 'd_ch3', 'a_ch2'}
Two_Analog: {'a_ch1', 'a_ch2'}
Analog1: {'a_ch1'}
Analog2: {'a_ch2'}
A1_M1_M2_A2: {'a_ch1', 'd_ch2', 'a_ch2', 'd_ch1'}
A1_A2_M3_M4: {'d_ch4', 'd_ch3', 'a_ch1', 'a_ch2'}


#### set_mode(), get_mode()

In [50]:
myawg.set_mode('C')
myawg.get_mode()

'C'

In [46]:
myawg.set_mode('T')
myawg.get_mode()

'T'

In [48]:
myawg.set_mode('G')
myawg.get_mode()

'G'

In [49]:
myawg.set_mode('S')
myawg.get_mode()

'S'

#### set/get_sample_rate()

In [51]:
myawg.set_sample_rate(15e6)

15000000.0

In [52]:
myawg.set_sample_rate(-15e6)

10000000.0

In [53]:
myawg.set_interleave(True)
myawg.set_sample_rate(23e9)

23000000000.0

In [54]:
myawg.set_interleave(False)
myawg.set_sample_rate(23e9)

12000000000.0

#### set/get_analog_level()

In [63]:
ampl_res, offset_res = myawg.set_analog_level(
    amplitude={'a_ch1': 0.6, 'a_ch2': 0.9},
    offset={'a_ch1': 0, 'a_ch2': 0}  # Offset in not supported for options [02] and [06]
)

for ch_key in ampl_res:
    print('{}: amp={}, offset={}'.format(ch_key, ampl_res[ch_key], offset_res[ch_key]))

a_ch1: amp=0.6, offset=0.0
a_ch2: amp=0.9, offset=0.0


In [67]:
ampl_res, offset_res = myawg.set_analog_level(
    amplitude={'a_ch1': 1.5, 'a_ch2': 0.9},
    offset={'a_ch1': 0, 'a_ch2': 0}  # Offset in not supported for options [02] and [06]
)

for ch_key in ampl_res:
    print('{}: amp={}, offset={}'.format(ch_key, ampl_res[ch_key], offset_res[ch_key]))

a_ch1: amp=1.0, offset=0.0
a_ch2: amp=0.9, offset=0.0


In [72]:
ampl_res, offset_res = myawg.set_analog_level(
    amplitude={'a_ch1': 0.95, 'a_ch2': 0.85, 'AABBCC1': 0.5},
    offset={'a_ch1': 0, 'a_ch2': 0}  # Offset in not supported for options [02] and [06]
)

for ch_key in ampl_res:
    print('{}: amp={}, offset={}'.format(ch_key, ampl_res[ch_key], offset_res[ch_key]))

a_ch1: amp=0.95, offset=0.0
a_ch2: amp=0.85, offset=0.0


#### set/get_digital_level()

In [1]:
low_res, high_res = myawg.set_digital_level(
    low={'d_ch1': -0.2, 'd_ch2': -0.1, 'd_ch3': 0.1, 'd_ch4': 0.2},
    high={'d_ch1': 0.8, 'd_ch2': 0.9, 'd_ch3': 1.1, 'd_ch4': 1.2}
)

for ch_key in low_res:
    print('{}: low={}, high={}'.format(ch_key, low_res[ch_key], high_res[ch_key]))

d_ch1: low=-0.2, high=0.8
d_ch2: low=-0.1, high=0.9
d_ch3: low=0.1, high=1.1
d_ch4: low=0.2, high=1.2


In [80]:
low_res, high_res = myawg.set_digital_level(
    low={'d_ch1': 0.3, 'd_ch2': 0.1},
    high={'d_ch3': 0.6, 'd_ch4': 0.7}
)

for ch_key in low_res:
    print('{}: low={}, high={}'.format(ch_key, low_res[ch_key], high_res[ch_key]))

d_ch1: low=0.3, high=0.8
d_ch2: low=0.1, high=0.9
d_ch3: low=0.1, high=0.6
d_ch4: low=0.2, high=0.7


In [95]:
# NOTICE: if low and high values are too far apart (max difference is 1.4 V),
# the last-setted param is set to the specified value, but the first-setted one is pulled to
# meet the max distance restriction

low_res, high_res = myawg.set_digital_level(
    low={'d_ch1': -0.5},
    high={'d_ch1': 1.2}
)

for ch_key in low_res:
    print('{}: low={}, high={}'.format(ch_key, low_res[ch_key], high_res[ch_key]))

d_ch1: low=-0.2, high=1.2
d_ch2: low=0.1, high=0.9
d_ch3: low=0.1, high=0.6
d_ch4: low=0.2, high=0.7


#### set/get_active_channels()

In [5]:
myawg._get_all_channels()

['a_ch1', 'a_ch2', 'd_ch1', 'd_ch2', 'd_ch3', 'd_ch4']

In [4]:
ch_st_dict = myawg.get_active_channels()
for ch_name, ch_state  in ch_st_dict.items():
    print('{}: {}'.format(ch_name, ch_state))

a_ch1: False
a_ch2: False
d_ch1: False
d_ch2: False
d_ch3: False
d_ch4: False


In [9]:
myawg.set_active_channels(
    {'a_ch1': True, 
     'a_ch2': True,
     'd_ch1': True,
     'd_ch2': True,
     'd_ch3': False,
     'd_ch4': False    
    }
)

ch_st_dict = myawg.get_active_channels()
for ch_name, ch_state  in ch_st_dict.items():
    print('{}: {}'.format(ch_name, ch_state))

a_ch1: True
a_ch2: True
d_ch1: True
d_ch2: True
d_ch3: False
d_ch4: False


In [16]:
myawg.set_active_channels(
    {'a_ch1': True, 
     'a_ch2': True,
     'd_ch1': True,
     'd_ch2': True,
     'd_ch3': True,
     'd_ch4': True,
    }
)

myawg.set_active_channels(
    {'a_ch1': True, 
     'a_ch2': False,
     'd_ch1': True,
     'd_ch2': True,
     'd_ch3': True,
     'd_ch4': True    
    }
)

ch_st_dict = myawg.get_active_channels()
for ch_name, ch_state  in ch_st_dict.items():
    print('{}: {}'.format(ch_name, ch_state))

a_ch1: True
a_ch2: True
d_ch1: True
d_ch2: True
d_ch3: True
d_ch4: True


`Transition to ({'d_ch3', 'd_ch1', 'a_ch1', 'd_ch4', 'd_ch2'}) active channels is not allowed according to constraints.
set_active_channels() aborted. Channel states not changed`

In [17]:
myawg.set_active_channels(
    {'a_ch1': True, 
     'a_ch2': True,
     'd_ch1': True,
     'd_ch2': True,
     'd_ch3': True,
     'd_ch4': True,
    }
)

myawg.set_active_channels(
    {'a_ch1': True, 
     'a_ch2': False,
     'd_ch1': True,
     'd_ch2': True,
     'd_ch3': False,
     'd_ch4': False,
     'd_ch5': False
    }
)

ch_st_dict = myawg.get_active_channels()
for ch_name, ch_state  in ch_st_dict.items():
    print('{}: {}'.format(ch_name, ch_state))

a_ch1: True
a_ch2: True
d_ch1: True
d_ch2: True
d_ch3: True
d_ch4: True


`Trying to (de)activate channels that are not present in AWG.
Setting of channel activation aborted.`

In [19]:
myawg.set_active_channels(
    {'a_ch1': False, 
     'a_ch2': False,
     'd_ch1': False,
     'd_ch2': False,
     'd_ch3': False,
     'd_ch4': False,
    }
)

myawg.set_active_channels(
    {'a_ch1': True}
)

ch_st_dict = myawg.get_active_channels()
for ch_name, ch_state  in ch_st_dict.items():
    print('{}: {}'.format(ch_name, ch_state))

a_ch1: True
a_ch2: False
d_ch1: False
d_ch2: False
d_ch3: False
d_ch4: False


In [21]:
myawg.set_active_channels(
    {'a_ch1': True, 
     'a_ch2': False,
     'd_ch1': False,
     'd_ch2': False,
     'd_ch3': False,
     'd_ch4': False,
    }
)

myawg.get_active_channels(['a_ch1', 'd_ch4'])

{'a_ch1': True, 'd_ch4': False}

In [22]:
myawg.set_active_channels(
    {'a_ch1': True, 
     'a_ch2': False,
     'd_ch1': False,
     'd_ch2': False,
     'd_ch3': False,
     'd_ch4': False,
    }
)

myawg.get_active_channels(['a_ch1', 'd_ch4', 'd_ch5'])

{'a_ch1': True, 'd_ch4': False}

`Channel "d_ch5" is not available in AWG.
Ignore channel state request for this entry.`

#### pulser_on/off()

##### _get_channel_state()

In [34]:
ch_st_dict = myawg._get_channel_state()
for ch_name, ch_state  in ch_st_dict.items():
    print('{}: {}'.format(ch_name, ch_state))

a_ch1: True
d_ch2: False
d_ch1: False
a_ch2: True
d_ch4: True
d_ch3: True


In [35]:
ch_st_dict = myawg._get_channel_state(['a_ch1', 'd_ch2', 'd_ch5'])
for ch_name, ch_state  in ch_st_dict.items():
    print('{}: {}'.format(ch_name, ch_state))

a_ch1: True
d_ch2: False


`Channel "d_ch5" is not available in AWG.
Ignore channel state request for this entry.`

##### _activate_channels()

In [28]:
myawg.set_active_channels(
    {'a_ch1': True,
     'd_ch1': True,
     'd_ch2': True,
     'a_ch2': True,
     'd_ch3': False,
     'd_ch4': False,
    }
)

myawg._activate_channels()

ch_st_dict = myawg._get_channel_state()
for ch_name, ch_state  in ch_st_dict.items():
    print('{}: {}'.format(ch_name, ch_state))

a_ch1: True
d_ch2: True
d_ch1: True
a_ch2: True
d_ch4: False
d_ch3: False


In [18]:
# Analog Channel 1 has a wevoform loaded
# Manually deleted waveform from Analog Channel 2 (via AWG's screen)

myawg.set_active_channels(
    {'a_ch1': True,
     'd_ch1': True,
     'd_ch2': True,
     'a_ch2': True,
     'd_ch3': False,
     'd_ch4': False,
    }
)

myawg._activate_channels()

ch_st_dict = myawg._get_channel_state()
for ch_name, ch_state  in ch_st_dict.items():
    print('{}: {}'.format(ch_name, ch_state))

a_ch1: True
d_ch2: True
d_ch1: True
a_ch2: False
d_ch4: False
d_ch3: False


```
AWG7122C error: 6000 "Other error; E11506 - OUTPUT2:STATE ON
-----------
Activation of channels by _activate_channels() failed
The expected state: {'a_ch1': True, 'a_ch2': True, 'd_ch1': True, 'd_ch2': True, 'd_ch3': False, 'd_ch4': False}
The obtained state: {'a_ch1': True, 'a_ch2': False, 'd_ch1': True, 'd_ch2': True, 'd_ch3': False, 'd_ch4': False}

```

##### pulser_on/off()

In [37]:
myawg.set_active_channels(
    {'a_ch1': True,
     'd_ch1': True,
     'd_ch2': True,
     'a_ch2': True,
     'd_ch3': False,
     'd_ch4': False,
    }
)

myawg.pulser_on()

2

In [38]:
myawg.pulser_off()
    
ch_st_dict = myawg.get_active_channels()
for ch_name, ch_state  in ch_st_dict.items():
    print('{}: {}'.format(ch_name, ch_state))

a_ch1: True
a_ch2: True
d_ch1: True
d_ch2: True
d_ch3: False
d_ch4: False


In [39]:
myawg.set_active_channels(
    {'a_ch1': True,
     'd_ch1': True,
     'd_ch2': True,
     'a_ch2': False,
     'd_ch3': False,
     'd_ch4': False,
    }
)

myawg.pulser_on()

2

In [40]:
myawg.pulser_off()
    
ch_st_dict = myawg.get_active_channels()
for ch_name, ch_state  in ch_st_dict.items():
    print('{}: {}'.format(ch_name, ch_state))

a_ch1: True
a_ch2: False
d_ch1: True
d_ch2: True
d_ch3: False
d_ch4: False


#### set/get_interleave()

In [41]:
myawg.set_interleave(state=True)

True

In [42]:
myawg.set_interleave(False)

False

#### set_lowpass_filter()   [not awailable on this AWG]

In [44]:
myawg.query('OUTPUT1:FILTER:LPASS:FREQUENCY?')

Traceback (most recent call last):
  File "C:\Users\Lukin SiV\qudi\logic\jupyterkernel\qzmqkernel.py", line 795, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-44-e158c0ec888f>", line 1, in <module>
    myawg.query('OUTPUT1:FILTER:LPASS:FREQUENCY?')
  File "C:\Users\Lukin SiV\qudi\hardware\awg\tektronix_awg7k.py", line 184, in query
    answer = self.awg.query(question)
  File "C:\Anaconda\envs\qudi\lib\site-packages\pyvisa\resources\messagebased.py", line 564, in query
    return self.read()
  File "C:\Anaconda\envs\qudi\lib\site-packages\pyvisa\resources\messagebased.py", line 413, in read
    message = self._read_raw().decode(enco)
  File "C:\Anaconda\envs\qudi\lib\site-packages\pyvisa\resources\messagebased.py", line 386, in _read_raw
    chunk, status = self.visalib.read(self.session, size)
  File "C:\Anaconda\envs\qudi\lib\site-packages\pyvisa\ctwrapper\functions.py", line 1584, in read
    ret = library.viRead(session, buffer, count, byr

`OUTPut[n]:FILTer[:LPASs]:FREQuency`
This command and query sets or returns the low-pass filter frequency of the
filter. INFinity is same as Through (no filter). This command is not available on
instruments with option 02 or option 06

In [45]:
myawg.set_lowpass_filter(a_ch=1, cutoff_freq=100)

`Low-pass filter is not available for options [02] and [06]. set_lowpass_filter() request was ignored`